In [1]:
from IPython.display import Image

----------------
## PANDAS - Pivot and crosstab
------------


In [2]:
import pandas as pd
import numpy as np

In [3]:
df_pivot = pd.read_excel(r"D:\MYLEARN\datasets\sales-funnel.xlsx")
df_pivot.head()    

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [4]:
df_pivot.shape

(17, 8)

In [5]:
df_pivot.sort_values(['Manager', 'Rep'])

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won
5,218895,Kulas Inc,Daniel Hilton,Debra Henley,CPU,2,40000,pending
6,218895,Kulas Inc,Daniel Hilton,Debra Henley,Software,1,10000,presented
7,412290,Jerde-Hilpert,John Smith,Debra Henley,Maintenance,2,5000,pending
8,740150,Barton LLC,John Smith,Debra Henley,CPU,1,35000,declined
9,141962,Herman LLC,Cedric Moss,Fred Anderson,CPU,2,65000,won


In [6]:
df_pivot.dtypes

Account      int64
Name        object
Rep         object
Manager     object
Product     object
Quantity     int64
Price        int64
Status      object
dtype: object

In [7]:
df_pivot['Status'].value_counts()

presented    6
pending      4
won          4
declined     3
Name: Status, dtype: int64

In [8]:
df_pivot["Status"] = df_pivot["Status"].astype("category")

df_pivot["Status"].cat.set_categories(["won", "pending", "presented", "declined"], inplace=True)

In [9]:
df_pivot['Status'].value_counts()

presented    6
pending      4
won          4
declined     3
Name: Status, dtype: int64

In [10]:
# simplest pivot table must have a dataframe and an index . In this case, let’s use the Name as our index.

pd.pivot_table(df_pivot, index=["Name"])

,Account,Price,Quantity
Name,,,
Barton LLC,740150,35000,1.000000
"Fritsch, Russel and Anderson",737550,35000,1.000000
Herman LLC,141962,65000,2.000000
Jerde-Hilpert,412290,5000,2.000000
"Kassulke, Ondricka and Metz",307599,7000,3.000000
Keeling LLC,688981,100000,5.000000
Kiehn-Spinka,146832,65000,2.000000
Koepp Ltd,729833,35000,2.000000
Kulas Inc,218895,25000,1.500000


You can have multiple indexes as well. 

In fact, most of the pivot_table args can take multiple values via a list.

In [11]:
pd.pivot_table(df_pivot, index=["Manager", "Rep"])

Account         Price  Quantity
Manager       Rep                                            
Debra Henley  Craig Booker   720237.0  20000.000000  1.250000
              Daniel Hilton  194874.0  38333.333333  1.666667
              John Smith     576220.0  20000.000000  1.500000
Fred Anderson Cedric Moss    196016.5  27500.000000  1.250000
              Wendy Yule     614061.5  44250.000000  3.000000

You can see that the pivot table is smart enough to start aggregating the data and summarizing it by grouping the reps with their managers. 

the Account and Quantity columns aren’t really useful. Let’s remove it by explicitly defining the columns we care about using the values field.

In [12]:
pd.pivot_table(df_pivot, index=["Manager", "Rep"], values=["Price"])

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000

The price column automatically averages the data but we can do a count or a sum.

In [13]:
pd.pivot_table(df_pivot, 
               index=["Manager","Rep"], 
               values=["Price"], 
               aggfunc=np.sum)

Price
Manager       Rep                  
Debra Henley  Craig Booker    80000
              Daniel Hilton  115000
              John Smith      40000
Fred Anderson Cedric Moss    110000
              Wendy Yule     177000

In [14]:
pd.pivot_table(df_pivot, 
               index=["Manager","Rep"], 
               values=["Price"],
               aggfunc=[np.sum, np.mean, len])

sum          mean   len
                              Price         Price Price
Manager       Rep                                      
Debra Henley  Craig Booker    80000  20000.000000     4
              Daniel Hilton  115000  38333.333333     3
              John Smith      40000  20000.000000     2
Fred Anderson Cedric Moss    110000  27500.000000     4
              Wendy Yule     177000  44250.000000     4

If we want to see sales broken down by the products, the __columns__ variable allows us to define one or more columns.

The aggregation functions are applied to the values you list.

In [15]:
pd.pivot_table(df_pivot,
               index  =["Manager", "Rep"],
               values =["Price"],
               columns=["Product"],
               aggfunc=[np.sum])

sum                             
                                Price                             
Product                           CPU Maintenance Monitor Software
Manager       Rep                                                 
Debra Henley  Craig Booker    65000.0      5000.0     NaN  10000.0
              Daniel Hilton  105000.0         NaN     NaN  10000.0
              John Smith      35000.0      5000.0     NaN      NaN
Fred Anderson Cedric Moss     95000.0      5000.0     NaN  10000.0
              Wendy Yule     165000.0      7000.0  5000.0      NaN

Add Quantity to the values list.

In [31]:
pd.pivot_table(df_pivot,
               index   =["Manager","Rep"],
               values  =["Price","Quantity"],
               columns =["Product"],
               aggfunc =[np.sum],
               fill_value=0)

sum                                        \
                              Price                              Quantity   
Product                         CPU Maintenance Monitor Software      CPU   
Manager       Rep                                                           
Debra Henley  Craig Booker    65000        5000       0    10000        2   
              Daniel Hilton  105000           0       0    10000        4   
              John Smith      35000        5000       0        0        1   
Fred Anderson Cedric Moss     95000        5000       0    10000        3   
              Wendy Yule     165000        7000    5000        0        7   

                                                          
                                                          
Product                     Maintenance Monitor Software  
Manager       Rep                                         
Debra Henley  Craig Booker            2       0        1  
              Daniel Hilton           0       0        1  
              John Smith              2       0        0  
Fred Anderson Cedric Moss             1       0        1  
              Wendy Yule              3       2        0

Remove Product from the columns and add to the index .

In [16]:
pd.pivot_table(df_pivot,
               index   =["Manager","Rep","Product"],
               values  =["Price","Quantity"],
               aggfunc =[np.sum],
               fill_value=0)

sum         
                                          Price Quantity
Manager       Rep           Product                     
Debra Henley  Craig Booker  CPU           65000        2
                            Maintenance    5000        2
                            Software      10000        1
              Daniel Hilton CPU          105000        4
                            Software      10000        1
              John Smith    CPU           35000        1
                            Maintenance    5000        2
Fred Anderson Cedric Moss   CPU           95000        3
                            Maintenance    5000        1
                            Software      10000        1
              Wendy Yule    CPU          165000        7
                            Maintenance    7000        3
                            Monitor        5000        2

Now, what if I want to see some totals? margins=True does that for us.

In [17]:
pd.pivot_table(df_pivot,
               index  =["Manager","Rep","Product"],
               values =["Price","Quantity"],
               aggfunc=[np.sum,np.mean],
               fill_value=0,
               margins=True)

sum                   mean  \
                                          Price Quantity         Price   
Manager       Rep           Product                                      
Debra Henley  Craig Booker  CPU           65000        2  32500.000000   
                            Maintenance    5000        2   5000.000000   
                            Software      10000        1  10000.000000   
              Daniel Hilton CPU          105000        4  52500.000000   
                            Software      10000        1  10000.000000   
              John Smith    CPU           35000        1  35000.000000   
                            Maintenance    5000        2   5000.000000   
Fred Anderson Cedric Moss   CPU           95000        3  47500.000000   
                            Maintenance    5000        1   5000.000000   
                            Software      10000        1  10000.000000   
              Wendy Yule    CPU          165000        7  82500.000000   
                            Maintenance    7000        3   7000.000000   
                            Monitor        5000        2   5000.000000   
All                                      522000       30  30705.882353   

                                                   
                                         Quantity  
Manager       Rep           Product                
Debra Henley  Craig Booker  CPU          1.000000  
                            Maintenance  2.000000  
                            Software     1.000000  
              Daniel Hilton CPU          2.000000  
                            Software     1.000000  
              John Smith    CPU          1.000000  
                            Maintenance  2.000000  
Fred Anderson Cedric Moss   CPU          1.500000  
                            Maintenance  1.000000  
                            Software     1.000000  
              Wendy Yule    CPU          3.500000  
                            Maintenance  3.000000  
                            Monitor      2.000000  
All                                      1.764706

Analysis at the manager level. 

Notice how the status is ordered based on our earlier category definition.

In [18]:
pd.pivot_table(df_pivot,
               index  =["Manager","Status"],
               values =["Price"],
               aggfunc=[np.sum],
               fill_value=0,
               margins=True)

sum
                          Price
Manager       Status           
Debra Henley  won         65000
              pending     50000
              presented   50000
              declined    70000
Fred Anderson won        172000
              pending      5000
              presented   45000
              declined    65000
All                      522000

A really handy feature is the ability to pass a dictionary to the aggfunc so you can perform different functions on each of the values you select. 

In [19]:
pd.pivot_table(df_pivot,
               index  =["Manager","Status"],
               columns=["Product"],
               values =["Quantity","Price"],
               aggfunc={"Quantity":len, "Price":np.sum},
               fill_value=0)

Price                              Quantity  \
Product                     CPU Maintenance Monitor Software      CPU   
Manager       Status                                                    
Debra Henley  won         65000           0       0        0        1   
              pending     40000       10000       0        0        1   
              presented   30000           0       0    20000        1   
              declined    70000           0       0        0        2   
Fred Anderson won        165000        7000       0        0        2   
              pending         0        5000       0        0        0   
              presented   30000           0    5000    10000        1   
              declined    65000           0       0        0        1   

                                                      
Product                 Maintenance Monitor Software  
Manager       Status                                  
Debra Henley  won                 0       0        0  
              pending             2       0        0  
              presented           0       0        2  
              declined            0       0        0  
Fred Anderson won                 1       0        0  
              pending             1       0        0  
              presented           0       1        1  
              declined            0       0        0

You can provide a list of aggfunctions to apply to each value too:

In [20]:
pd.pivot_table(df_pivot,
               index   =["Manager","Status"],
               columns =["Product"],
               values  =["Quantity","Price"],
               aggfunc ={"Quantity":len,"Price":[np.sum,np.mean]},
               fill_value=0)

Price                                       \
                          mean                                  sum   
Product                    CPU Maintenance Monitor Software     CPU   
Manager       Status                                                  
Debra Henley  won        65000           0       0        0   65000   
              pending    40000        5000       0        0   40000   
              presented  30000           0       0    10000   30000   
              declined   35000           0       0        0   70000   
Fred Anderson won        82500        7000       0        0  165000   
              pending        0        5000       0        0       0   
              presented  30000           0    5000    10000   30000   
              declined   65000           0       0        0   65000   

                                                     Quantity              \
                                                          len               
Product                 Maintenance Monitor Software      CPU Maintenance   
Manager       Status                                                        
Debra Henley  won                 0       0        0        1           0   
              pending         10000       0        0        1           2   
              presented           0       0    20000        1           0   
              declined            0       0        0        2           0   
Fred Anderson won              7000       0        0        2           1   
              pending          5000       0        0        0           1   
              presented           0    5000    10000        1           0   
              declined            0       0        0        1           0   

                                          
                                          
Product                 Monitor Software  
Manager       Status                      
Debra Henley  won             0        0  
              pending         0        0  
              presented       0        2  
              declined        0        0  
Fred Anderson won             0        0  
              pending         0        0  
              presented       1        1  
              declined        0        0

#### PANDAS CROSSTAB

Example

In [21]:
# Define the headers since the data does not have any
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

In [23]:
# Read in the CSV file and convert "?" to NaN
df_raw = pd.read_csv(r"D:\MYLEARN\DATASETS\imports-85.data",
                     header=None, 
                     names=headers, 
                     na_values="?" )

# Define a list of models that we want to review
models = ["toyota","nissan","mazda", "honda", "mitsubishi", "subaru", "volkswagen", "volvo"]

# Create a copy of the data with only the top 8 manufacturers
df_auto = df_raw[df_raw.make.isin(models)].copy()

In [24]:
df_auto.shape

(128, 26)

In [25]:
pd.crosstab(df_auto.make, df_auto.body_style)

body_style,convertible,hardtop,hatchback,sedan,wagon
make,,,,,
honda,0,0,7,5,1
mazda,0,0,10,7,0
mitsubishi,0,0,9,4,0
nissan,0,1,5,9,3
subaru,0,0,3,5,4
toyota,1,3,14,10,4
volkswagen,1,0,1,9,1
volvo,0,0,0,8,3


In [19]:
# step 1
df_auto.groupby(['make', 'body_style'])['body_style'].count()

make        body_style 
honda       hatchback       7
            sedan           5
            wagon           1
mazda       hatchback      10
            sedan           7
mitsubishi  hatchback       9
            sedan           4
nissan      hardtop         1
            hatchback       5
            sedan           9
            wagon           3
subaru      hatchback       3
            sedan           5
            wagon           4
toyota      convertible     1
            hardtop         3
            hatchback      14
            sedan          10
            wagon           4
volkswagen  convertible     1
            hatchback       1
            sedan           9
            wagon           1
volvo       sedan           8
            wagon           3
Name: body_style, dtype: int64

In [20]:
# step 2
df_auto.groupby(['make', 'body_style'])['body_style'].count().unstack()

body_style,convertible,hardtop,hatchback,sedan,wagon
make,,,,,
honda,NaN,NaN,7.0,5.0,1.0
mazda,NaN,NaN,10.0,7.0,NaN
mitsubishi,NaN,NaN,9.0,4.0,NaN
nissan,NaN,1.0,5.0,9.0,3.0
subaru,NaN,NaN,3.0,5.0,4.0
toyota,1.0,3.0,14.0,10.0,4.0
volkswagen,1.0,NaN,1.0,9.0,1.0
volvo,NaN,NaN,NaN,8.0,3.0


In [21]:
# step 3
df_auto.groupby(['make', 'body_style'])['body_style'].count().unstack().fillna(0)

body_style,convertible,hardtop,hatchback,sedan,wagon
make,,,,,
honda,0.0,0.0,7.0,5.0,1.0
mazda,0.0,0.0,10.0,7.0,0.0
mitsubishi,0.0,0.0,9.0,4.0,0.0
nissan,0.0,1.0,5.0,9.0,3.0
subaru,0.0,0.0,3.0,5.0,4.0
toyota,1.0,3.0,14.0,10.0,4.0
volkswagen,1.0,0.0,1.0,9.0,1.0
volvo,0.0,0.0,0.0,8.0,3.0


In [22]:
pd.crosstab(df_auto.make, df_auto.body_style, margins=True)

body_style,convertible,hardtop,hatchback,sedan,wagon,All
make,,,,,,
honda,0,0,7,5,1,13
mazda,0,0,10,7,0,17
mitsubishi,0,0,9,4,0,13
nissan,0,1,5,9,3,18
subaru,0,0,3,5,4,12
toyota,1,3,14,10,4,32
volkswagen,1,0,1,9,1,12
volvo,0,0,0,8,3,11
All,2,4,49,57,16,128


In [23]:
# data sumarization - percentage of time each combination occurs. 
# This can be accomplished using the normalize parameter:
pd.crosstab(df_auto.make, df_auto.body_style, normalize=True)

body_style,convertible,hardtop,hatchback,sedan,wagon
make,,,,,
honda,0.000000,0.000000,0.054688,0.039062,0.007812
mazda,0.000000,0.000000,0.078125,0.054688,0.000000
mitsubishi,0.000000,0.000000,0.070312,0.031250,0.000000
nissan,0.000000,0.007812,0.039062,0.070312,0.023438
subaru,0.000000,0.000000,0.023438,0.039062,0.031250
toyota,0.007812,0.023438,0.109375,0.078125,0.031250
volkswagen,0.007812,0.000000,0.007812,0.070312,0.007812
volvo,0.000000,0.000000,0.000000,0.062500,0.023438


- This table shows us that 2.3% of the total population are Toyota hardtops and 6.25% are Volvo sedans.

In [24]:
# The normalize parameter is allows us to perform summary on just the columns or rows. 
# For example, if we want to see how the body styles are distributed across makes:

pd.crosstab(df_auto.make, df_auto.body_style, normalize='columns')

body_style,convertible,hardtop,hatchback,sedan,wagon
make,,,,,
honda,0.0,0.00,0.142857,0.087719,0.0625
mazda,0.0,0.00,0.204082,0.122807,0.0000
mitsubishi,0.0,0.00,0.183673,0.070175,0.0000
nissan,0.0,0.25,0.102041,0.157895,0.1875
subaru,0.0,0.00,0.061224,0.087719,0.2500
toyota,0.5,0.75,0.285714,0.175439,0.2500
volkswagen,0.5,0.00,0.020408,0.157895,0.0625
volvo,0.0,0.00,0.000000,0.140351,0.1875
